In [7]:
import regex as re
import pandas as pd
import nltk
import numpy as np
from nltk.corpus import stopwords
from nltk.stem.wordnet import WordNetLemmatizer
import string
import gensim
from gensim import corpora
import pyLDAvis.gensim_models
nltk.download('wordnet')
nltk.download('omw-1.4')

[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\kevin\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package omw-1.4 to
[nltk_data]     C:\Users\kevin\AppData\Roaming\nltk_data...
[nltk_data]   Package omw-1.4 is already up-to-date!


True

In [9]:
def importing_Lyrics_df():
    df = pd.read_csv('azlyrics-scraper/azlyrics_lyrics_19.csv',on_bad_lines='skip')
    return df

In [11]:
def pre_processing(text):
    stop_list = []
    stop_list.extend(stopwords.words('english'))
    stop_list.extend(
        ['yo','dont','nigga','uh', 'got', 'oh', 'im', 'na', 'from', 'subject', 're', 'edu', 'use', 'not', 'would', 'say', 'could', '_', 'be',
         'know', 'good', 'go', 'get', 'ah', 'bout','yeah','le','ayy','u','bitch','eh','wa',
         'do', 'done', 'try', 'many', 'some', 'nice', 'thank', 'think', 'see', 'rather', 'easy', 'easily', 'lot',
         'lack', 'make', 'want', 'seem', 'run', 'need', 'even', 'right', 'line', 'even', 'also', 'may', 'take', 'come'])
    exclude = set(string.punctuation)
    lemma = WordNetLemmatizer()

    text = text.rstrip()
    normalized = " ".join(lemma.lemmatize(i) for i in text.split())
    punc_free = ''.join(i for i in normalized if i not in exclude)
    stop_free = " ".join([i for i in punc_free.lower().split() if((i not in stop_list) and (not i.isdigit()))])
    return stop_free

In [13]:

def modelling(clean_text):
    dictionary = corpora.Dictionary(clean_text)
    corpus = [dictionary.doc2bow(text) for text in text_clean]
    ldamodel = gensim.models.ldamodel.LdaModel(corpus, num_topics=5, id2word=dictionary, passes=5)
    topics = ldamodel.print_topics(num_words=5)
    return (dictionary, corpus, ldamodel, topics)

In [15]:
#Importing the CSV file
df = importing_Lyrics_df()

In [16]:
#Cleaning the data
text_clean=[]
for text in df['LYRICS']:
    text_clean.append(pre_processing(text).split())

In [19]:
#Modelling
dictionary, corpus, ldamodel, topics = modelling(text_clean)
for topic in topics:
    print(topic)

(0, '0.013*"love" + 0.013*"time" + 0.010*"never" + 0.009*"way" + 0.008*"youre"')
(1, '0.009*"love" + 0.009*"back" + 0.007*"issa" + 0.006*"like" + 0.005*"nae"')
(2, '0.010*"wanna" + 0.007*"youre" + 0.007*"like" + 0.007*"gonna" + 0.005*"away"')
(3, '0.016*"like" + 0.010*"love" + 0.008*"baby" + 0.007*"feel" + 0.007*"let"')
(4, '0.027*"like" + 0.012*"aint" + 0.012*"cant" + 0.009*"fuck" + 0.008*"shit"')


In [20]:
lda_display = pyLDAvis.gensim_models.prepare(ldamodel, corpus, dictionary, sort_topics=False)
pyLDAvis.display(lda_display)

D:\4.Semester\Machine Learning\venv\lib\site-packages\pyLDAvis\_prepare.py:247: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only.
  by='saliency', ascending=False).head(R).drop('saliency', 1)


In [21]:
def get_topic_details(ldamodel, corpus):
    topic_details_df = pd.DataFrame()
    for i, row in enumerate(ldamodel[corpus]):
        row = sorted(row, key=lambda x: (x[1]), reverse=True)
        for j, (topic_num, prop_topic) in enumerate(row):
            if j == 0:  # => dominant topic
                wp = ldamodel.show_topic(topic_num)
                topic_details_df = topic_details_df.append(pd.Series([topic_num, prop_topic]),
                ignore_index=True)
    topic_details_df.columns = ['Dominant_Topic', '% Score']
    return topic_details_df

In [22]:
contents = pd.DataFrame({'Original Text': text_clean})
topic_details = pd.concat([get_topic_details(ldamodel, corpus), contents], axis=1)

# Create flag for text highest associated with topic 3
dom_tp = topic_details['Dominant_Topic']
topic_details['flag'] = np.where((dom_tp == 1.0) | (dom_tp == 4.0), 1, 0)
print(topic_details.head())

C:\Users\kevin\AppData\Local\Temp\ipykernel_16664\2135296541.py:8: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  topic_details_df = topic_details_df.append(pd.Series([topic_num, prop_topic]),
C:\Users\kevin\AppData\Local\Temp\ipykernel_16664\2135296541.py:8: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  topic_details_df = topic_details_df.append(pd.Series([topic_num, prop_topic]),


   Dominant_Topic   % Score  \
0             4.0  0.997914   
1             4.0  0.618445   
2             4.0  0.996423   
3             3.0  0.794323   
4             4.0  0.996455   

                                       Original Text  flag  
0  [one, grape, street, gslide, sweet, shay, shay...     1  
1  [gonna, put, suit, tie, steady, speakin, work,...     1  
2  [kill, kill, radio, killer, music, drug, deale...     1  
3  [lil, steady, speaking, low, project, wall, ta...     0  
4  [x, loaded, bro, never, church, prayer, lost, ...     1  
